In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'a-price aok-align-center'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'a-price a-text-price'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [11]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=keyboards&crid=3JG4WYT3TZ0VO&sprefix=keyboards%2Caps%2C485&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
        href = link.get('href')
        if href:
            full_url = href if href.startswith('http') else "https://www.amazon.com" + href
            links_list.append(full_url)

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    

In [12]:
links_list

['https://www.amazon.com/AmazonBasics-Matte-Keyboard-QWERTY-Layout/dp/B07WJ5D3H4/ref=sr_1_1?crid=3JG4WYT3TZ0VO&dib=eyJ2IjoiMSJ9.poeU91llN73R2NK_qzZaT_eiWM3M6J-8MrYUjfmzWeCIG96qcFVOmvBb83cO6A5hdvCiZSFVAbliFgalMu-ZNsOwTakkpQT7KYXtyfn20X9VWsQwWiSi4TZQprDl03N9kwVEoMYQyCpP3u5ce3Syszufp4uOIWR7kytgCQDPu4vF8bHwU2lmG1s1yWDld7JRTav1xcwyJJL5i11lUiCPoO88wLARM9yqacnJU2BOzZ8.oDYTVU1K27MM7wVncynfzN1fRokT_Z-PlBKpEajdGDA&dib_tag=se&keywords=keyboards&qid=1722603490&sprefix=keyboards%2Caps%2C485&sr=8-1',
 'https://www.amazon.com/AULA-F99-Mechanical-Bluetooth-Swappable/dp/B0CLLHSWRL/ref=sr_1_2?crid=3JG4WYT3TZ0VO&dib=eyJ2IjoiMSJ9.poeU91llN73R2NK_qzZaT_eiWM3M6J-8MrYUjfmzWeCIG96qcFVOmvBb83cO6A5hdvCiZSFVAbliFgalMu-ZNsOwTakkpQT7KYXtyfn20X9VWsQwWiSi4TZQprDl03N9kwVEoMYQyCpP3u5ce3Syszufp4uOIWR7kytgCQDPu4vF8bHwU2lmG1s1yWDld7JRTav1xcwyJJL5i11lUiCPoO88wLARM9yqacnJU2BOzZ8.oDYTVU1K27MM7wVncynfzN1fRokT_Z-PlBKpEajdGDA&dib_tag=se&keywords=keyboards&qid=1722603490&sprefix=keyboards%2Caps%2C485&sr=8-2',
 'https://www.amaz

In [15]:
for link in links_list:
        new_webpage = requests.get(link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
amazon_df = pd.DataFrame.from_dict(d)
amazon_df['title'].replace('', np.nan, inplace=True)
amazon_df = amazon_df.dropna(subset=['title'])
amazon_df.to_csv("reviews_data.csv", header=True, index=False)

In [16]:
amazon_df

,title,price,rating,reviews,availability
0,Amazon Basics Low-Profile Wired USB Keyboard w...,,4.5 out of 5 stars,"6,919 ratings",In Stock
1,"AULA F99 Wireless Mechanical Keyboard,Tri-Mode...",,4.8 out of 5 stars,404 ratings,In Stock
2,Womier S-K80 75% Keyboard with Color Multimedi...,,4.6 out of 5 stars,385 ratings,In Stock
3,Trueque Wireless Keyboard with 7 Colored Backl...,,4.1 out of 5 stars,22 ratings,In Stock
4,Trueque Wireless Keyboard with 7 Colored Backl...,,4.1 out of 5 stars,22 ratings,In Stock
5,"Snpurdiri 60% Percent Gaming Keyboard, Ergonom...",,4.4 out of 5 stars,"1,130 ratings",In Stock
6,Mechanical Keyboard Gaming Keyboard | Blue Swi...,,4.3 out of 5 stars,"1,546 ratings",In Stock
7,Rapoo V500Pro Full-Size Mechanical Gaming Keyb...,,4.3 out of 5 stars,64 ratings,In Stock
8,"60% Wired RGB Gaming Mechanical Keyboard, 61 K...",,4.6 out of 5 stars,52 ratings,Not Available
9,MageGee Portable 60% Mechanical Gaming Keyboar...,,4.4 out of 5 stars,"7,546 ratings",In Stock
